# 기존 v4와 df_brd_w_prd_v2 상품 개수 비교

In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
#$from tqdm import tnrange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [3]:
df_brd_w_prd_v2 = pd.read_csv("df_brd_w_prd_v2.csv")
df_brd_w_prd_v2 = df_brd_w_prd_v2.iloc[:, 1:]

v4 = pd.read_csv("V4_DF_ind_set.csv")
v4 = v4.iloc[:, 1:]

### 각각 브랜드명 set list

In [3]:
df_brd_w_prd_v2_brd_lst = list(df_brd_w_prd_v2["brand_name"]) #df_brd_w_prd_v2에는 브랜드 링크를 기준으로 이미 중복을 제거하였기 때문에 set하지 않음 (브랜드 명이 중복된 경우 있음)

v4_brd_lst = list(set(v4["브랜드명"]))

In [4]:
print("df_brd_w_prd_v2의 unique한 브랜드 개수: ", len(df_brd_w_prd_v2_brd_lst))
print("v4의 unique한 브랜드 개수: ", len(v4_brd_lst))

df_brd_w_prd_v2의 unique한 브랜드 개수:  8056
v4의 unique한 브랜드 개수:  1471


---
v4에는 있는데 df_brd_w_prd_v2에는 없는 브랜드 : v4_not_in_df_brd_w_prd_v2

In [5]:
v4_not_in_df_brd_w_prd_v2 = []

for i in range(len(v4_brd_lst)):
    if v4_brd_lst[i] not in df_brd_w_prd_v2_brd_lst:
        v4_not_in_df_brd_w_prd_v2.append(v4_brd_lst[i])

In [6]:
len(v4_not_in_df_brd_w_prd_v2)

7

In [7]:
v4_not_in_df_brd_w_prd_v2

['JM솔루션', 'W.피부연구소', 'DS래보래토리즈', 'DTRT', 'Dr.G', 'JHP 코스메틱', 'TMR 코스메틱스']

---
### [브랜드 리스트에 아예 포함되어 있지 않은 브랜드]
- DU care
- DS래보래토리즈
- 차우주
- DTRT
- Dr.G
- DRUNK ELEPHANT
- DPC
- DHC
- Dr.C60
- 더 스킨라파
- D'ena
---
### v4와 df_brd_w_prd_v2에서 같은 브랜드이지만 이름이 다르게 되어있는 브랜드 (영어와 한글이 섞여 있는 브랜드)
v4 / df_brd_w_prd_v2
- CJ제일제당 / 
- DS래보래토리즈 / DS LAB
- LC 프로페셔널 / LC PROFESSIONAL
- JHP 코스메틱 / JHP Cosmetic
- TNL / True Life
- JCK 메이크업 브러쉬 / JCK
- CU스킨 / CUSKIN
- 563랩 / 563LAB
- TMR 코스메틱스 / TMR cosmetics
- KVD 비건뷰티 / KVD VEGUN BEAUTY
- DK 코스메틱 / DK COSMETIC
- W.피부연구소 / W.SKIN LABORSTORY
- JM솔루션 / JMsolution

# v4 브랜드별 상품 개수 파악

In [8]:
v4_grp_cnt = pd.DataFrame(v4.groupby(["브랜드명"]).count()["제품_link"])

In [9]:
v4_grp_cnt = v4_grp_cnt.rename(columns = {'제품_link': 'product count (v4)',}, inplace = False)

In [10]:
v4_grp_cnt

,product count (v4)
브랜드명,
16BRAND,95
23 years old,74
3CE,289
76N1,8
AHC,437
...,...
히말라야,60
히스테모,2
히즈클린,2


# df_brd_w_prd_v2 브랜드별 상품 개수 파악

In [11]:
df_brd_w_prd_v2_grp_cnt = pd.DataFrame({'브랜드명': [], 'product count (df_brd_w_prd_v2)': []})

In [12]:
import ast

for _ in range(len(df_brd_w_prd_v2)):
    df_brd_w_prd_v2.loc[_, 'product_name'] = literal_eval(df_brd_w_prd_v2.loc[_, 'product_name'])
    df_brd_w_prd_v2.loc[_, 'product_link'] = literal_eval(df_brd_w_prd_v2.loc[_, 'product_link'])

In [13]:
for i in range(len(df_brd_w_prd_v2)):
    df_brd_w_prd_v2_grp_cnt.loc[i] = [df_brd_w_prd_v2.loc[i, "brand_name"], int(len(df_brd_w_prd_v2.loc[i, "product_link"]))]

In [14]:
df_brd_w_prd_v2_grp_cnt

,브랜드명,product count (df_brd_w_prd_v2)
0,가가알로에,1.0
1,가그린,15.0
2,가네보,12.0
3,가드맨,1.0
4,가드미,2.0
...,...,...
8051,5Star4A,1.0
8052,5일동안,3.0
8053,76N1,8.0
8054,87발렌타인,1.0


In [15]:
grp_cnt = v4_grp_cnt.merge(df_brd_w_prd_v2_grp_cnt, how="inner", on="브랜드명")

In [16]:
grp_cnt

,브랜드명,product count (v4),product count (df_brd_w_prd_v2)
0,16BRAND,95,95.0
1,23 years old,74,74.0
2,3CE,289,289.0
3,76N1,8,8.0
4,AHC,437,438.0
...,...,...,...
1461,히말라야,60,61.0
1462,히스테모,2,2.0
1463,히즈클린,2,2.0
1464,힐링버드,89,90.0


In [306]:
for i in range(len(list(df_brd_w_prd_v2.loc[df_brd_w_prd_v2["brand_name"]=="올리브영", 'product_name'])[0])):
    if list(df_brd_w_prd_v2.loc[df_brd_w_prd_v2["brand_name"]=="올리브영", 'product_name'])[0][i].strip() not in list(v4.loc[v4["브랜드명"]=="올리브영", '제품명'].st):
        print(list(df_brd_w_prd_v2.loc[df_brd_w_prd_v2["brand_name"]=="올리브영", 'product_name'])[0][i])

[단종]  앞머리 헤어롤
[단종]  슬림헤드 면도기
[단종]  올리브영 멀티 코튼 [5겹] 화장솜
[단종]  4D 모션브러쉬
[단종]  1/2 화장솜
[단종]  버블 거품망
[단종]  실키 화장솜
[단종]  엠보싱 화장솜
[단종]  클렌징퍼프
[단종]  실키마인 화장솜
[단종]  밀착퍼프
[단종]  집게 헤어롤
[단종]  알루미늄 헤어롤
[단종]  아이 메이크업 브러쉬 세트
[단종]  립브러쉬
[단종]  브러쉬드라이클린스폰지
[단종]  곤약 클렌징 퍼프
[단종]  고급 압축 퍼프 (원형2P)
[단종]  프리미엄 화장솜
[단종]  고급 해면 스폰지
[단종]  WHP 수분강화 숯 클렌징폼
[단종]  모찌퍼프
[단종]  극세사 큐티글로브
[단종]  라운드 화장솜
[단종]  다이아스펀지
[단종]  순면 화장솜
[단종]  솜털 세안 모공 브러시
[단종]  쿠션 퍼프
[단종]  티슈 화장솜
[단종]  스킨티슈 화장솜
[단종]  끙가퍼프
[단종]  일반 화장솜
[단종]  티트리 피부진정 숯 클렌징 폼
[단종]  HDP 모공청정 숯 클렌징 폼


In [17]:
grp_cnt.to_csv("grp_cnt.csv")

In [18]:
v4_more_than_df_brd_w_prd_v2 = []
df_brd_w_prd_v2_more_than_v4 = []
for i in range(len(grp_cnt)):
    if grp_cnt.loc[i, 'product count (v4)'] > grp_cnt.loc[i, 'product count (df_brd_w_prd_v2)']:
        v4_more_than_df_brd_w_prd_v2.append(grp_cnt.loc[i, '브랜드명'])
    elif grp_cnt.loc[i, 'product count (v4)'] < grp_cnt.loc[i, 'product count (df_brd_w_prd_v2)']:
        df_brd_w_prd_v2_more_than_v4.append(grp_cnt.loc[i, '브랜드명'])
    else:
        continue

In [23]:
print("상품개수가 v4에서 더 많은 브랜드 개수: ", len(v4_more_than_df_brd_w_prd_v2))

상품개수가 v4에서 더 많은 브랜드 개수:  20


In [24]:
print("상품개수가 df_brd_w_prd_v2에서 더 많은 브랜드 개수: ", len(df_brd_w_prd_v2_more_than_v4))

상품개수가 df_brd_w_prd_v2에서 더 많은 브랜드 개수:  227


### 상품개수가 v4에서 더 많은 33개 브랜드의 df_brd_w_prd_v2의 제품 유실 경로 파악

In [25]:
grp_cnt.loc[grp_cnt["브랜드명"]=="라이온", :]

,브랜드명,product count (v4),product count (df_brd_w_prd_v2)
262,라이온,23,21.0


In [4]:
lst = []
for i in range(len(list(df_brd_w_prd_v2.loc[df_brd_w_prd_v2["brand_name"]=="라뮤즈", 'product_name'])[0])):
    lst.append(list(df_brd_w_prd_v2.loc[df_brd_w_prd_v2["brand_name"]=="라뮤즈", 'product_name'])[0][i].replace(' ', ''))

In [28]:
cnt = 0
for i in range(len(list(v4.loc[v4["브랜드명"]=="라뮤즈", '제품명']))):
    if list(v4.loc[v4["브랜드명"]=="라뮤즈", '제품명'])[i].replace(' ', '') not in lst:
        print(list(v4.loc[v4["브랜드명"]=="라뮤즈", '제품명'])[i])
        cnt += 1
print(cnt)

페어 아크네 클렌징 폼
냉감테라피 아이스 페이스 메이크오버 시트
2


In [11]:
v4_brd_lst = set(list(v4.loc[:, ["브랜드명"]]))

In [29]:
#v4에서 상품이 중복된 경우의 수 파악
dup_cnt = 0
for brd in v4_more_than_df_brd_w_prd_v2:
    if len(list(v4.loc[v4["브랜드명"]== brd, "제품명"])) != len(list(set(list(v4.loc[v4["브랜드명"]== brd, "제품명"])))):
        dup_cnt += 1
print("v4에서 상품이 중복된 브랜드 개수: ", dup_cnt)

v4에서 상품이 중복된 브랜드 개수:  10


# v4보다 적게 크롤링 된 브랜드 다시 크롤링

In [322]:
v4_more_than_df_brd_w_prd_v2

['누리숲',
 '더블유드레스룸',
 '라벨영',
 '라이온',
 '라포랩',
 '레시피',
 '리엔케이',
 '마몽드',
 '마쉐리',
 '메디플라워',
 '미쉘메르시에',
 '미즈온',
 '벨',
 '보니앤제이',
 '뷰티피플',
 '비쉬',
 '세이어스',
 '에버에센스',
 '엘프 코스메틱스',
 '올리브영',
 '자이모겐',
 '제이에스티나 뷰티',
 '착한팩토리',
 '카이',
 '코츠비',
 '티스',
 '티스',
 '티타니아',
 '파에온',
 '팜스테이',
 '프레쉬',
 '프레쉬',
 '프리엔제']

In [395]:
dict_prd_link_tmp = {}
dict_prd_name_tmp = defaultdict(list)

In [397]:
for brd in v4_more_than_df_brd_w_prd_v2:
    brd_name = brd

    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    socket.setdefaulttimeout(30)

    url = 'https:/www.glowpick.com/searches/keywords'
    wd.get(url)
    time.sleep(5)

    wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()
    wd.maximize_window()
    time.sleep(3)

    brd_input = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
    brd_input.send_keys(brd_name)
    brd_input.send_keys(Keys.ENTER)
    time.sleep(3)

    def parsing_html_with_class(parsing_info = '.products'):
        html = wd.page_source
        soup = soup = BeautifulSoup(html, "html.parser")
        interm_r = soup.select(parsing_info)
        return interm_r

    try:
        while True:
            prd_link = []
            prd_name = []

            product_lst = parsing_html_with_class()
            product_len = len(re.findall('<div class="products__product product" (.*?)', str(product_lst[0])))

            #while wd.find_elements_by_class_name('details__brand')[product_len].text[:len(brd_name)] == brd_name:

            element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(product_len - 1))
            wd.execute_script("arguments[0].scrollIntoView(true);", element)
            time.sleep(2)

            product_lst_2 = parsing_html_with_class()

            if product_lst == product_lst_2:
                for i in range(product_len):
                    if (re.sub(r'\([^)]*\)', '', wd.find_elements_by_class_name('details__brand')[i].text).strip()) == brd_name:
                        code = re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0]))[i][1].split('-')[-1]
                        link = 'https://www.glowpick.com/products/' + code
                        prd_link.append(link)
                        dict_prd_name_tmp[brd_name].append(wd.find_elements_by_class_name('details__product')[i].text)
                dict_prd_link_tmp[brd_name] = prd_link
                break

    except NoSuchElementException:
        for i in range(product_len):
            if (re.sub(r'\([^)]*\)', '', wd.find_elements_by_class_name('details__brand')[i].text).strip()) == brd_name:
                code = re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0]))[i][1].split('-')[-1]
                link = 'https://www.glowpick.com/products/' + code
                prd_link.append(link)
                dict_prd_name_tmp[brd_name].append(wd.find_elements_by_class_name('details__product')[i].text)
        dict_prd_link_tmp[brd_name] = prd_link  



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1500.55 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.15 (KHTML, like Gecko) Chrome/24.0.1295.0 Safari/537.15


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577


...

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577


Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


KeyboardInterrupt: 

In [31]:
import pickle

with open("dict_prd_name_tmp.txt", "rb") as f:
    dict_prd_name_tmp_ = pickle.load(f)
    
with open("dict_prd_link_tmp.txt", "rb") as f:
    dict_prd_link_tmp_ = pickle.load(f)

# v4와 df_brd_w_prd_v2 상품명 비교

In [32]:
brd_lst = grp_cnt["브랜드명"]

In [39]:
dict_diff = {}
for brd in brd_lst:
    v2_lst = []
    v4_lst = []
    v2_not_v4 = []
    v4_not_v2 = []
    for j in range(len(list(df_brd_w_prd_v2.loc[df_brd_w_prd_v2["brand_name"]== brd, 'product_name'])[0])):
        v2_lst.append(list(df_brd_w_prd_v2.loc[df_brd_w_prd_v2["brand_name"]== brd, 'product_name'])[0][j].replace(' ', ''))
    #print(v2_lst)
        
    for k in range(len(list(v4.loc[v4["브랜드명"]== brd, '제품명']))):
        v4_lst.append(list(v4.loc[v4["브랜드명"]== brd, '제품명'])[k].replace(' ', ''))
    #print(v4_lst)
        
    for _ in range(len(v2_lst)):
        if v2_lst[_] not in v4_lst: #v2에는 있는데 v4에 없는 제품
            v2_not_v4.append(v2_lst[_])
            
    for _ in range(len(v4_lst)):
        if v4_lst[_] not in v2_lst: #v4에는 있는데 v2에 없는 제품
            v4_not_v2.append(v4_lst[_])
    dict_diff[brd] = [v2_not_v4, v4_not_v2]

In [40]:
import pickle

with open("dict_diff.txt", "wb") as f:
    pickle.dump(dict_diff, f)

In [41]:
cnt1 = 0  #상품 정보 업데이트(변경 + 추가)된 경우 카운트
cnt2 = 0 #상품 정보가 삭제된 경우(=df_brd_w_prd_v2에서 잡히지 않는 경우) 카운트
cnt3 = 0 #상품 정보가 추가된 경우(=v4에서 잡히지 않는 경우)

df_missing = []
v4_missing = []
update = []

for i in range(len(list(dict_diff.items()))):
    
    if list(dict_diff.items())[i][1][0] != [] and list(dict_diff.items())[i][1][1] != []: #상품 정보 업데이트(변경 + 추가)된 경우
        update.append(list(dict_diff.items())[i])
        cnt1 += len(list(dict_diff.items())[i][1][0])
        
    if list(dict_diff.items())[i][1][0] == [] and list(dict_diff.items())[i][1][1] != []: #상품 정보 삭제된 경우 (= df에서 유실된 경우)
        df_missing.append(list(dict_diff.items())[i])
        cnt2 += len(list(dict_diff.items())[i][1][1])
        
    if list(dict_diff.items())[i][1][0] != [] and list(dict_diff.items())[i][1][1] == []: #v4에서 상품 정보 유실만된 경우
        v4_missing.append(list(dict_diff.items())[i])
        cnt3 += len(list(dict_diff.items())[i][1][0])

In [42]:
import pickle

with open("df_missing.txt", "wb") as f:
    pickle.dump(df_missing, f)
    
with open("v4_missing.txt", "wb") as f:
    pickle.dump(v4_missing, f)
    
with open("update.txt", "wb") as f:
    pickle.dump(update, f)

In [48]:
print("상품 정보 업데이트(변경 + 추가)된 경우", cnt1)
print("상품 정보가 삭제된 경우(=df_brd_w_prd_v2에서 잡히지 않는 경우): ", cnt2)
print("상품 정보가 추가된 경우(=v4에서 잡히지 않는 경우)", cnt3)

상품 정보 업데이트(변경 + 추가)된 경우 1171
상품 정보가 삭제된 경우(=df_brd_w_prd_v2에서 잡히지 않는 경우):  7
상품 정보가 추가된 경우(=v4에서 잡히지 않는 경우) 489
